In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [3]:
data_path = r"data\旭诺CTA三号净值20250115.xlsx"
strategy = "量化CTA"
fund_name = "旭诺CTA三号"
benchmark_code = "NH0100.NHF"
benchmark_name = "南华商品指数"
threshold = -0.05

demo = NavResearch(data_path,strategy,fund_name,benchmark_code,benchmark_name,threshold)
df_nav, df_return, df_drawdown = demo.get_data()
tables = demo.get_analysis_table()

In [11]:
def multi_fund_comparison(tables,fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame(columns=[2019,2020,2021,2022,2023,2024])
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024]
    for year in years:
        data3[f"{year}收益"] = data4.loc[data4["分年度业绩"] == year, f"{fund_name}_收益"].values
        data3[f"{year}最大回撤"] = data4.loc[data4["分年度业绩"] == year, f"{fund_name}_最大回撤"].values
    data = pd.concat([data1,data2,data3], axis=1)
    data.drop(columns=["基准指数","整体业绩"], inplace=True)
    return data

In [12]:
multi_fund_comparison(tables,fund_name)

,基金产品,净值起始日期,净值结束日期,单位净值,累计净值,复权净值,总收益率,年化收益率,最大回撤,年化波动率,...,2020收益,2020最大回撤,2021收益,2021最大回撤,2022收益,2022最大回撤,2023收益,2023最大回撤,2024收益,2024最大回撤
0,旭诺CTA三号,2019-06-03,2025-01-10,3.8368,3.8368,3.8368,283.68%,27.08%,-18.17%,19.90%,...,81.47%,-15.70%,22.29%,-13.08%,-7.58%,-18.17%,17.79%,-3.93%,63.81%,-8.42%


In [6]:
tables[0]

,基金产品,基准指数,净值起始日期,净值结束日期,单位净值,累计净值,复权净值
0,旭诺CTA三号,南华商品指数,2019-06-03,2025-01-10,3.8368,3.8368,3.8368


In [7]:
tables[1].head(1)

,整体业绩,总收益率,年化收益率,最大回撤,年化波动率,夏普比率
0,绝对收益,283.68%,27.08%,-18.17%,19.90%,1.26


In [8]:
tables[2]

,分年度业绩,旭诺CTA三号_收益,旭诺CTA三号_最大回撤,南华商品指数_收益,南华商品指数_最大回撤,超额收益
0,2019,-5.85%,-6.38%,6.82%,-4.21%,-12.67%
1,2020,81.47%,-15.70%,7.52%,-16.07%,73.95%
2,2021,22.29%,-13.08%,20.94%,-10.76%,1.35%
3,2022,-7.58%,-18.17%,19.66%,-22.72%,-27.24%
4,2023,17.79%,-3.93%,6.21%,-10.98%,11.58%
5,2024,63.81%,-8.42%,-2.08%,-14.10%,65.89%
6,2025,2.97%,-0.94%,0.69%,0.00%,2.29%
